In [1]:
import torch
from pytorch_quantization import tensor_quant

from torch import nn, outer

from pytorch_quantization import tensor_quant
import pytorch_quantization.nn as quant_nn
import pytorch_quantization
pytorch_quantization.__version__
from tqdm import tqdm

In [2]:
from datasets.yolo import YOLO
from engine.detector import Detector
from model.squeezedet import SqueezeDet, SqueezeDetWithLoss
from utils.config import Config
from utils.model import load_model
from PIL import Image
import numpy as np
import argparse
import os
from utils.misc import init_env

In [3]:
args = {'_': '_'}
cfg = Config().parse(args)

init_env(cfg) 
 
cfg.mode = 'eval'
cfg.exp_id = "test_lpd_sqdet_trt_calib"
cfg.dataset = 'lpr'
cfg.load_model = '../exp/alpr_det.pth' 
cfg.batch_size = 1
cfg.arch = 'squeezedet'
cfg.num_workers = 0
cfg.save_intervals = 1
cfg.debug = 2
cfg.load_pretrained = True

def update_exp_dir(cfg, exp_id):
    cfg.save_dir = os.path.join(cfg.exp_dir, exp_id)
    os.makedirs(cfg.save_dir, exist_ok=True)
    cfg.log_file = os.path.join(cfg.save_dir, 'training_logs.txt')
    os.remove(cfg.log_file) if os.path.exists(cfg.log_file) else None
    cfg.debug_dir = os.path.join(cfg.save_dir, 'debug')

update_exp_dir(cfg, cfg.exp_id)

Config().print(cfg)


arch                           squeezedet
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
data_dir                       /home/hazen/data/datasets
dataset                        lpr
debug                          2
debug_dir                      /home/hazen/data/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/debug
device                         cpu
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /home/hazen/data/SqueezeDet-PyTorch/exp
exp_id                         test_lpd_sqdet_trt_calib
flip_prob                      0.5
forbid_resize                  False
gpus                           [0]
gpus_str                       0
grad_norm                      0.5
keep_top_k                     64
load_model                     ../exp/alpr_det.pth
load_pretrained                True
log_file                       /home/hazen/data/SqueezeDet-PyTorc

In [4]:
from engine.detector import Detector
from model.squeezedet import SqueezeDetWithLoss
from utils.model import load_model
from utils.misc import load_dataset

In [5]:
dataset = load_dataset(cfg.dataset)('val', cfg)
cfg = Config().update_dataset_info(cfg, dataset)
Config().print(cfg)

anchors                        [[  8.   8.   3.   3.]
 [  8.   8.   6.   5.]
 [  8.   8.   9.   5.]
 ...
 [120. 120.  10.   9.]
 [120. 120.  10.  12.]
 [120. 120.  15.   8.]]
anchors_per_grid               6
arch                           squeezedet
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
class_names                    0
data_dir                       /home/hazen/data/datasets
dataset                        lpr
debug                          2
debug_dir                      /home/hazen/data/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/debug
device                         cpu
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /home/hazen/data/SqueezeDet-PyTorch/exp
exp_id                         test_lpd_sqdet_trt_calib
flip_prob                      0.5
forbid_resize                  False
gpus                           [0]
gpus_str     

### Vanilla Eval

In [6]:
from utils.misc import init_env
init_env(cfg)
from eval import eval
eval(cfg)

anchors                        [[  8.   8.   3.   3.]
 [  8.   8.   6.   5.]
 [  8.   8.   9.   5.]
 ...
 [120. 120.  10.   9.]
 [120. 120.  10.  12.]
 [120. 120.  15.   8.]]
anchors_per_grid               6
arch                           squeezedet
batch_size                     1
bbox_loss_weight               20.0
chunk_sizes                    [32]
class_loss_weight              1.0
class_names                    0
data_dir                       /home/hazen/data/datasets
dataset                        lpr
debug                          2
debug_dir                      /home/hazen/data/SqueezeDet-PyTorch/exp/test_lpd_sqdet_trt_calib/debug
device                         cpu
drift_prob                     1.0
dropout_prob                   0
exp_dir                        /home/hazen/data/SqueezeDet-PyTorch/exp
exp_id                         test_lpd_sqdet_trt_calib
flip_prob                      0.5
forbid_resize                  False
gpus                           [0]
gpus_str     

In [6]:
if cfg.mode=='eval':
        model = SqueezeDetWithLoss(cfg)
        model = load_model(model, cfg.load_model, cfg)


model.detect = True
detector = Detector(model, cfg)
print(detector.model)


loaded model ../exp/alpr_det.pth, epoch 268
state_dict: 
 dict_keys(['base.conv1.weight', 'base.conv1.bias', 'base.features.1.squeeze.weight', 'base.features.1.squeeze.bias', 'base.features.1.expand1x1.weight', 'base.features.1.expand1x1.bias', 'base.features.1.expand3x3.weight', 'base.features.1.expand3x3.bias', 'base.features.2.squeeze.weight', 'base.features.2.squeeze.bias', 'base.features.2.expand1x1.weight', 'base.features.2.expand1x1.bias', 'base.features.2.expand3x3.weight', 'base.features.2.expand3x3.bias', 'base.features.4.squeeze.weight', 'base.features.4.squeeze.bias', 'base.features.4.expand1x1.weight', 'base.features.4.expand1x1.bias', 'base.features.4.expand3x3.weight', 'base.features.4.expand3x3.bias', 'base.features.5.squeeze.weight', 'base.features.5.squeeze.bias', 'base.features.5.expand1x1.weight', 'base.features.5.expand1x1.bias', 'base.features.5.expand3x3.weight', 'base.features.5.expand3x3.bias', 'base.features.7.squeeze.weight', 'base.features.7.squeeze.bias', '

In [12]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib
from pytorch_quantization.tensor_quant import QuantDescriptor

from absl import logging
logging.set_verbosity(logging.INFO)  # Disable logging as they are too noisy in notebook

In [13]:
quant_desc_input = QuantDescriptor(calib_method='histogram')
quant_nn.QuantConv2d.set_default_quant_desc_input(quant_desc_input)
quant_nn.QuantLinear.set_default_quant_desc_input(quant_desc_input)

In [14]:
from pytorch_quantization import quant_modules
quant_modules.initialize()

### Calibration Functions

In [ ]:
def collect_stats(model, data_loader, num_batches):
    """Feed data to the network and collect statistic"""

    # Enable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    # for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
    #     model(image.cuda())
    #     if i >= num_batches:
    #         break
    # since lpr dataloader return a dict, so writing it in this way
    for i, (batch) in tqdm(enumerate(data_loader), total=num_batches):
        for k in batch:
                if 'image_meta' not in k:
                    batch[k] = batch[k].to(device="cuda", non_blocking=True)
        x, _ = model(batch)
        if i >= num_batches:
            break

    # Disable calibrators
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()
            
def compute_amax(model, **kwargs):
    # Load calib result
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax(**kwargs)
#             print(F"{name:40}: {module}")
    model.cuda()

## Make Dataset and DataLoader

In [7]:
import os
import subprocess

import numpy as np
import skimage.io

from datasets.base import BaseDataset
from utils.boxes import generate_anchors
from PIL import Image
from torchvision.datasets.folder import default_loader


class LPR_CalibTRT(BaseDataset):
    def __init__(self, phase, cfg):
        super(LPR_CalibTRT, self).__init__(phase, cfg)

        # self.input_size = (128, 128)  # (height, width), both dividable by 16
        #this model for sadaia works with (256,256)
        self.input_size = (256, 256)  # (height, width), both dividable by 16
        self.class_names = ('0')
        # real_filtered mean and std
        # self.rgb_mean = np.array([94.87347, 96.89165, 94.70493], dtype=np.float32).reshape(1, 1, 3)
        # self.rgb_std = np.array([53.869507, 53.936283, 55.2807], dtype=np.float32).reshape(1, 1, 3)
        
        # real_filtered plus all_sites_seatbelt mean and std
        # self.rgb_mean = np.array([104.90631, 105.41336, 104.70162], dtype=np.float32).reshape(1, 1, 3)
        # self.rgb_std = np.array([50.69564, 49.60443, 50.158844], dtype=np.float32).reshape(1, 1, 3)

        self.rgb_mean = np.array([97.631615, 98.70732, 98.41285], dtype=np.float32).reshape(1, 1, 3)
        self.rgb_std = np.array([52.766678, 52.63513, 52.348827], dtype=np.float32).reshape(1, 1, 3)

        self.num_classes = len(self.class_names)
        self.class_ids_dict = {cls_name: cls_id for cls_id, cls_name in enumerate(self.class_names)}

        self.data_dir = os.path.join(cfg.data_dir, 'lpr_crop/merged_data')
        self.sample_ids, self.sample_set_path = self.get_sample_ids()
        self.grid_size = tuple(x // cfg.stride for x in self.input_size)  # anchors grid 
        # self.anchors_seed = np.array([[ 29, 17], [46, 32], [69, 52],
        #                                 [109, 68], [84, 127], [155, 106], 
        #                                 [255, 145], [183, 215], [371, 221]], dtype=np.float32) ## real_filtered anchors
        
        # self.anchors_seed = np.array( [[ 32, 20], [ 61, 42], [ 59, 97],
        #                                 [103, 66], [122, 114], [183, 96],
        #                                 [160, 152], [211, 201], [343, 205]], dtype=np.float32) ## real_filtered plus all_sites_seatbelt anchors

        # self.anchors_seed = np.array([[6, 5], [12, 10], [18, 10], [18, 18], [20, 24], [30, 15]], dtype=np.float32)
        self.anchors_seed = np.array([[3, 3], [6, 5], [9, 5], [10, 9], [10, 12], [15, 8]], dtype=np.float32)
        # self.anchors = generate_anchors(self.grid_size, self.input_size, self.anchors_seed)
        self.anchors = torch.load("../exp/alpr_det_anchors_256_exp_8.pt")
        self.anchors_per_grid = self.anchors_seed.shape[0]
        self.num_anchors = self.anchors.shape[0]

        self.results_dir = os.path.join(cfg.save_dir, 'results')

    def get_sample_ids(self):
        #a dirty duct tape to load preprocessing of val phase but load dataset for train phase for trt calib
        sample_set_name = 'train.txt' if self.phase == 'train' \
            else 'train.txt' if self.phase == 'val' \
            else 'trainval.txt' if self.phase == 'trainval' \
            else None

        sample_ids_path = os.path.join(self.data_dir, sample_set_name)
        with open(sample_ids_path, 'r') as fp:
            sample_ids = fp.readlines()
        sample_ids = tuple(x.strip() for x in sample_ids)

        return sample_ids, sample_ids_path

    def load_image(self, index):
        image_id = self.sample_ids[index]
        image_path = os.path.join(self.data_dir, 'images', image_id + '.png')
        image = default_loader(image_path)
        if image.mode == 'L':
            image = image.convert('RGB')
        image = np.array(image).astype(np.float32)
        # image = skimage.io.imread(image_path).astype(np.float32)
        return image, image_id

    def load_annotations(self, index):
        ann_id = self.sample_ids[index]
        ann_path = os.path.join(self.data_dir, 'labels', ann_id + '.txt')
        with open(ann_path, 'r') as fp:
            annotations = fp.readlines()

        annotations = [ann.strip().split(' ') for ann in annotations]
        class_ids, boxes = [], []
        for ann in annotations:
            if ann[0] not in self.class_names:
                continue
            class_ids.append(self.class_ids_dict[ann[0]])
            box = [float(x) for x in ann[4:8]]
            # if box[2] <= 0:
            #     box[2] = 0.00001
            # if box[3] <= 0:
            #     box[3] = 0.00001
            boxes.append(box)

        class_ids = np.array(class_ids, dtype=np.int16)
        boxes = np.array(boxes, dtype=np.float32)
        if len(boxes):
            return class_ids, boxes
        boxes = None
        return class_ids, boxes

    # ========================================
    #                evaluation
    # ========================================

    def save_results(self, results):
        txt_dir = os.path.join(self.results_dir, 'data')
        os.makedirs(txt_dir, exist_ok=True)

        for res in results:
            txt_path = os.path.join(txt_dir, res['image_meta']['image_id'] + '.txt')
            if 'class_ids' not in res:
                with open(txt_path, 'w') as fp:
                    fp.write('')
                continue

            num_boxes = len(res['class_ids'])
            with open(txt_path, 'w') as fp:
                for i in range(num_boxes):
                    class_name = self.class_names[res['class_ids'][i]].lower()
                    score = res['scores'][i]
                    bbox = res['boxes'][i, :]
                    line = '{} -1 -1 0 {:.2f} {:.2f} {:.2f} {:.2f} 0 0 0 0 0 0 0 {:.3f}\n'.format(
                            class_name, *bbox, score)
                    fp.write(line)

    def evaluate(self):
        kitti_eval_tool_path = os.path.join(self.cfg.root_dir, 'src/utils/kitti-eval/cpp/evaluate_object')
        cmd = '{} {} {} {} {}'.format(kitti_eval_tool_path,
                                      os.path.join(self.data_dir),
                                      self.sample_set_path,
                                      self.results_dir,
                                      len(self.sample_ids))

        print (cmd)
        status = subprocess.call(cmd, shell=True)

        aps = {}
        for class_name in self.class_names:
            map_path = os.path.join(self.results_dir, 'stats_{}_ap.txt'.format(class_name.lower()))
            if os.path.exists(map_path):
                with open(map_path, 'r') as f:
                    lines = f.readlines()
                _aps = [float(line.split('=')[1].strip()) for line in lines]
            else:
                _aps = [0., 0., 0.]

            aps[class_name + '_easy'] = _aps[0]
            aps[class_name + '_moderate'] = _aps[1]
            aps[class_name + '_hard'] = _aps[2]

        aps['mAP'] = sum(aps.values()) / len(aps)

        return aps

In [ ]:
lpr_calib_dataset = LPR_CalibTRT('val', cfg)
calib_data_loader = torch.utils.data.DataLoader(lpr_calib_dataset,
                                                  batch_size=cfg.batch_size,
                                                  num_workers=cfg.num_workers,
                                                  pin_memory=False)

In [ ]:
print (len(lpr_calib_dataset))

14068


### Compute stats and Calibrate on Different schemes

In [ ]:
num_batches = int(len(lpr_calib_dataset)/cfg.batch_size)
print (num_batches)

detector.model.eval()
detector.model.cuda()
with torch.no_grad():
    collect_stats(detector.model, calib_data_loader, num_batches=2000)
    # compute_amax(detector.model, method="percentile", percentile=99.99)

In [ ]:
print (detector.model)


In [ ]:
from datasets.lpr import LPR

eval_dataset = LPR('val', cfg)
eval_dataset.phase

'val'

In [ ]:
with torch.no_grad():
    for method in ["percentile", "mse", "entropy"]:
        percentile = 99.99
        print(F" ############  {method} calibration ######################")
        if(method == 'percentile'):
            compute_amax(detector.model, method=method, percentile = percentile)
        else:
            compute_amax(detector.model, method=method)
        update_exp_dir(cfg, f"trt_calib_lpd_sadaia_sqdet_{method}")
        results = detector.detect_dataset(eval_dataset, cfg)
        eval_dataset.save_results(results)
        aps = eval_dataset.evaluate()
        for k, v in aps.items():
                print('{:<20} {:.3f}'.format(k, v))
        torch.save(detector.model.state_dict(), os.path.join(cfg.save_dir, f"quant_lp_squeezedet_sadaia_{method}.pth"))

 ############  percentile calibration ######################
torch.Size([1, 3, 128, 128])


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
# print (detector.model)

### Add scales to the quantized state dict

In [ ]:
dict = torch.load("../exp/trt_calib_lpd_sadaia_sqdet_percentile/quant_lp_squeezedet_sadaia_percentile.pth")
print (dict.keys())



In [ ]:
# print (alpr)
def get_scales_input_per_tensor(amax, _num_bits = 8 , _unsigned = False):
    bound = (1 << (_num_bits - 1 + int(_unsigned))) - 1    
    zero_point_per_tensor = 0 #since symmertric quant
    scale_per_tensor = amax / bound
    return scale_per_tensor, zero_point_per_tensor

def get_scales_weight_per_channel(amax, _num_bits = 8, _unsigned = False):
    bound = (1 << (_num_bits - 1 + int(_unsigned))) - 1
    amax_sequeeze = amax.squeeze().detach()
    scale_per_channel =  amax_sequeeze/bound
    zero_point_per_channel = torch.zeros_like(scale_per_channel, dtype=torch.int32).data
    quant_axis = list(amax.shape).index(list(amax_sequeeze.shape)[0])
    return scale_per_channel, zero_point_per_channel


In [ ]:
lp_det_sadaia_dict_with_scales = {}

for key in dict:
    if key.find('_input_quantizer._amax') != -1:
        amax = dict[key]
        scales, zero_point = get_scales_input_per_tensor(amax)
        new_key = key + '.scales'
        lp_det_sadaia_dict_with_scales[new_key] = scales
    elif key.find('_weight_quantizer._amax') != -1:
        amax = dict[key]
        scales, zero_point = get_scales_weight_per_channel(amax)
        new_key = key + '.scales'
        lp_det_sadaia_dict_with_scales[new_key] = scales
    
    lp_det_sadaia_dict_with_scales[key] = dict[key]

print (lp_det_sadaia_dict_with_scales.keys())

dict_keys(['base.conv1.weight', 'base.conv1.bias', 'base.conv1._input_quantizer._amax.scales', 'base.conv1._input_quantizer._amax', 'base.conv1._weight_quantizer._amax.scales', 'base.conv1._weight_quantizer._amax', 'base.features.1.squeeze.weight', 'base.features.1.squeeze.bias', 'base.features.1.squeeze._input_quantizer._amax.scales', 'base.features.1.squeeze._input_quantizer._amax', 'base.features.1.squeeze._weight_quantizer._amax.scales', 'base.features.1.squeeze._weight_quantizer._amax', 'base.features.1.expand1x1.weight', 'base.features.1.expand1x1.bias', 'base.features.1.expand1x1._input_quantizer._amax.scales', 'base.features.1.expand1x1._input_quantizer._amax', 'base.features.1.expand1x1._weight_quantizer._amax.scales', 'base.features.1.expand1x1._weight_quantizer._amax', 'base.features.1.expand3x3.weight', 'base.features.1.expand3x3.bias', 'base.features.1.expand3x3._input_quantizer._amax.scales', 'base.features.1.expand3x3._input_quantizer._amax', 'base.features.1.expand3x3._

In [ ]:
torch.save(lp_det_sadaia_dict_with_scales, "../exp/quant_lp_det_sadaia_with_scales.pth")

In [106]:
from glob import glob
from os import *
from utils import image
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.transforms.functional import InterpolationMode

images_dir = "/home/hazen/data/SqueezeDet-PyTorch/data/test_trt"
convdet_dir = "/home/hazen/data/SqueezeDet-PyTorch/data/wrong_convdet/"
image_paths = glob(path.join(images_dir,"*"))
image_paths.sort()
for i, path in enumerate(image_paths):
        img = default_loader(path)
        # if img.mode == 'L':
        #         img = img.convert('RGB')
        print(np.array(img).shape)
        trt_xl = np.load(convdet_dir+path.split("/")[-1].split(".")[0]+".npy")
        x = np.array(img)-trt_xl
        print(np.max(x))
        # img = transforms.ToTensor()(img)
        # img = transforms.Resize((256,256),interpolation=InterpolationMode.NEAREST) (img)
        # img = transforms.Grayscale(num_output_channels=3) (img)
        
        # trt_xl = np.load(convdet_dir+path.split("/")[-1].split(".")[0]+".npy")
        # x = img.numpy()-trt_xl
        # print(np.max(x))
        # # img = img.permute(2, 0, 1)
        # # print(img.shape)
        # img = img.reshape(1,3,256,256)
        # trt_xl = trt_xl.reshape(1,3,256,256)
        # x = img.numpy()-trt_xl
        # # print("2",np.max(x))
        # img = img.cuda()
        
        # x,xl = model.base(img)
        # # xl = xl.reshape(36,16,16)
        # xl = xl.to('cpu').detach().numpy()
        # # print("TRT: ",trt_xl)
        # # print("PyTorch: ", xl)
        # #print(np.max(xl-trt_xl))

(600, 800, 3)
81.0
(600, 800, 3)
90.0
(600, 800, 3)
139.0
(600, 800, 3)
150.0
(600, 800, 3)
85.0
(600, 800, 3)
129.0
(600, 800, 3)
93.0
(600, 800, 3)
118.0
(600, 800, 3)
93.0
(600, 800, 3)
132.0
(600, 800, 3)
112.0
(600, 800, 3)
73.0


In [14]:
model.cuda()
x = torch.rand(size=(1,3,256,256)).to("cuda")
x,xl = model.base(x)
print(xl)

tensor([[[[-1.7525e-08, -6.0345e-08, -7.5839e-08,  ...,  6.9797e-08,
            1.1774e-07, -1.6262e-08],
          [ 8.8885e-08,  1.5105e-07, -7.9799e-08,  ...,  9.6999e-08,
            1.0943e-07,  8.8961e-08],
          [ 1.8245e-08,  7.4072e-08,  5.2735e-08,  ...,  1.0661e-07,
            3.4629e-08,  6.4640e-08],
          ...,
          [ 1.1090e-08, -2.8018e-08, -8.3166e-08,  ...,  2.2642e-08,
            1.9277e-07,  3.1397e-08],
          [ 6.4179e-09,  1.2052e-07,  5.0777e-08,  ...,  9.7679e-08,
            1.3850e-07,  8.2770e-08],
          [ 5.0847e-09, -5.7183e-08,  4.0392e-08,  ...,  4.4975e-08,
            6.7946e-08, -8.4506e-09]],

         [[-2.8623e+00, -2.8623e+00, -2.8623e+00,  ..., -2.8623e+00,
           -2.8623e+00, -2.8623e+00],
          [-2.8623e+00, -2.8623e+00, -2.8623e+00,  ..., -2.8623e+00,
           -2.8623e+00, -2.8623e+00],
          [-2.8623e+00, -2.8623e+00, -2.8623e+00,  ..., -2.8623e+00,
           -2.8623e+00, -2.8623e+00],
          ...,
     

In [13]:
model.base

SqueezeDetBase(
  (quant): QuantStub()
  (dequant): DeQuantStub()
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu1): ReLU(inplace=True)
  (features): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (1): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activation_1): ReLU(inplace=True)
      (activation_2): ReLU(inplace=True)
      (activation_3): ReLU(inplace=True)
      (float_functional_simple): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (2): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activa